# EJERCICIO DE DIFUSIÓN DE INFORMACIÓN
## Redes Sociales, Colaboración en Red

**Autores:**  
Martín Salinas Antón - martin.salinas@estudiante.uam.es  
Belén Vivas García - belen.vivas@estudiante.uam.es

In [1]:
import networkx as nx
import matplotlib.pyplot as plt

### Lectura de Datos

In [2]:
G = nx.read_graphml('data/juegoDtronos.graphml')

In [3]:
# Normalize weights into probabilities
weights = []
for edge in G.edges(data=True):
    # Get weight from graph and add it to the weight list
    weight = edge[2]['weight']
    weights.append(weight)

normalized_weights = []
# Get the maximum weight
max_weight = max(weights)
for w in weights:
    # Divide each weight by the maximum to normalize between 0 and 1
    w /= max_weight
    normalized_weights.append(w)

# Get a copy of the graph
G_norm = G
# Modify the weights
i = 0
for edge in G_norm.edges(data=True):
    edge[2]['weight'] = normalized_weights[i]
    i += 1

# Turn the graph into undirected
# G_u = nx.to_undirected(G_norm)

In [ ]:
# Max spread cascades - greedy algorithm
def max_spread_cascades_greedy(G, k):
    # Set of initially activated nodes
    s = set()


seeds = max_spread_cascades_greedy(G_norm, 5)
print(seeds)